# Write a Table of Images and Categorical Labels

In this example we will write a small table containing images of cats and dogs.
The table will have two visible columns: one for the image and one for the label.
In addition, we will add a "sample-weight" column, which will not be visible in the table rows.

In [ ]:
import os
from pathlib import Path
import tlc
from PIL import Image

## Collect lists of image paths and labels

In [ ]:
data_path = Path("../data/cats-and-dogs").absolute().as_posix()
dir2label = {"cats": 0, "dogs": 1}

image_files = []
labels = []

for dir_name in ["cats", "dogs"]:
    for file_name in os.listdir(f"{data_path}/{dir_name}"):
        image_files.append(f"{data_path}/{dir_name}/{file_name}")
        labels.append(dir2label[dir_name])


## Write a table using a TableWriter

In [ ]:
project_name = "image-classification-table"

table_writer = tlc.TableWriter(
    table_name="initial",
    dataset_name="cats-and-dogs",
    project_name=project_name,
    column_schemas={
        "image": tlc.ImagePath,
        "label": tlc.CategoricalLabel("label", ["cat", "dog"]),
        "weight": tlc.SampleWeightSchema(),
    },
    if_exists="overwrite",
)

for image, label in zip(image_files, labels):
    table_writer.add_row(
        {
            "image": image,
            "label": label,
            "weight": 1.0,
        }
    )

table = table_writer.finalize()

print(f"Created table with {len(table)} rows.")

print("\nFirst row (sample view):")
print(table[0])

print("\nFirst row (table view):")
print(table.table_rows[0])

In [ ]:
# Map the table so that it is compatible with typical classification models
def map_fn(sample):
    return (Image.open(sample["image"]), sample["label"])

table.map(map_fn)
table[0]  # Now the first row is a tuple with the image and the label